In [2]:
!pip install -qq roboflow ultralytics opencv-python torch torchvision easyocr pyyaml wandb

In [3]:
!pip install -U -qq ipywidgets

In [4]:
import torch
import torchvision.transforms as transforms
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import wandb
import yaml

from roboflow import Roboflow
from ultralytics import YOLO

2024-06-29 12:58:59.351656: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 12:58:59.351767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 12:58:59.480433: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
ROBOFLOW_APIKEY = 'API_KEY'
rf = Roboflow(api_key=f"{ROBOFLOW_APIKEY}")
project = rf.workspace().project("idn-license-plate-detection")
dataset_obb = project.version(1).download("yolov8-obb")

loading Roboflow workspace...
loading Roboflow project...


In [77]:
project.get_version_information()

[{'id': 'object-detection-shc88/idn-license-plate-detection/1',
  'name': 'idn-license-plate-v1',
  'created': 1719666125.683,
  'images': 1598,
  'splits': {'valid': 305, 'test': 196, 'train': 1097},
  'preprocessing': {'auto-orient': True},
  'augmentation': {},
  'exports': ['yolov8', 'yolov8-obb']}]

In [11]:
wandb.login(key="API_KEY")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [81]:
!cat {dataset.location}/data.yaml

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


names:
  0: license-plate
path: /kaggle/working/IDN-License-Plate-Detection-1
test: /kaggle/working/IDN-License-Plate-Detection-1/test/images
train: /kaggle/working/IDN-License-Plate-Detection-1/train/images
val: /kaggle/working/IDN-License-Plate-Detection-1/valid/images


In [80]:
yaml_path = os.path.join(dataset_obb.location, 'data.yaml')

# Read the existing YAML file
with open(yaml_path, 'r') as file:
    data = yaml.safe_load(file)

# Modify the YAML content
data['path'] = dataset_obb.location
data['train'] = os.path.join(dataset_obb.location, 'train/images')
data['val'] = os.path.join(dataset_obb.location, 'valid/images')
data['test'] = os.path.join(dataset_obb.location, 'test/images')
data['names'] = {0: 'license-plate'}

# Write the updated YAML content back to the file
with open(yaml_path, 'w') as file:
    yaml.safe_dump(data, file)

print("YAML file updated successfully.")

YAML file updated successfully.


In [19]:
model = YOLO("yolov8x-obb.yaml").load("yolov8x-obb.pt")

100%|██████████| 133M/133M [00:01<00:00, 96.5MB/s] 


Transferred 631/637 items from pretrained weights


In [20]:
import torch
print(torch.cuda.device_count())

2


In [ ]:
result = model.train(
    data=f"{dataset.location}/data.yaml", 
    epochs=100, 
    imgsz=640, 
    device=[0, 1], # use 2 GPU
    single_cls=True,
)

Ultralytics YOLOv8.2.46 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=yolov8x-obb.yaml, data=/kaggle/working/IDN-License-Plate-Detection-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fals

wandb: Currently logged in as: dzalhaqi (dzal). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240629_135319-3gqo5o52
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train3
wandb: ⭐️ View project at https://wandb.ai/dzal/YOLOv8
wandb: 🚀 View run at https://wandb.ai/dzal/YOLOv8/runs/3gqo5o52


Overriding model.yaml nc=80 with nc=1
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /kaggle/working/IDN-License-Plate-Detection-1/train/labels.cache... 1097 images, 0 backgrounds, 24 corrupt: 100%|██████████| 1097/1097 [00:00<?, ?it/s]


train: WARNING ⚠️ /kaggle/working/IDN-License-Plate-Detection-1/train/images/DataTrain101_jpg.rf.38be56aac2044be7e79f817e36308360.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0026]
train: WARNING ⚠️ /kaggle/working/IDN-License-Plate-Detection-1/train/images/DataTrain108_jpg.rf.b6dee06785157ab22f3edb81349a4a86.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0237      1.0343]
train: WARNING ⚠️ /kaggle/working/IDN-License-Plate-Detection-1/train/images/DataTrain121_jpg.rf.a5ad92f968b691ee4a5eb427707ffd31.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0238      1.0021]
train: WARNING ⚠️ /kaggle/working/IDN-License-Plate-Detection-1/train/images/DataTrain130_jpg.rf.670138cb087b0b0bc2a9fd37def3d86f.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0076]
train: WARNING ⚠️ /kaggle/working/IDN-License-Plate-Detection-1/train/images/DataTra

...

     50/100      7.84G     0.6083     0.4345       2.34          1        640: 100%|██████████| 68/68 [00:45<00:00,  1.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:06<00:00,  5.87it/s]


                   all        297        323      0.935      0.944      0.938      0.718

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     51/100      7.84G     0.5878     0.4311       2.35          2        640: 100%|██████████| 68/68 [00:45<00:00,  1.49it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:06<00:00,  5.66it/s]


                   all        297        323      0.924      0.944      0.943       0.71

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     52/100      7.73G     0.5878     0.4209      2.376         25        640:  56%|█████▌    | 38/68 [00:25<00:20,  1.46it/s]

In [112]:
# if training process is terminated
last_checkpoint = "runs/obb/train3/weights/last.pt" 

last_model = YOLO(last_checkpoint)
remaining_epochs = 100 - 51  # epoch total - epoch have been run

last_result = last_model.train(
    data=f"{dataset.location}/data.yaml", 
    epochs=remaining_epochs, 
    imgsz=640, 
    device=[0, 1],
    single_cls=True,
)

Ultralytics YOLOv8.2.46 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=runs/obb/train3/weights/last.pt, data=/kaggle/working/IDN-License-Plate-Detection-1/data.yaml, epochs=49, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=N

wandb: Currently logged in as: dzalhaqi (dzal). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240629_154637-iryqyb0e
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train4
wandb: ⭐️ View project at https://wandb.ai/dzal/YOLOv8
wandb: 🚀 View run at https://wandb.ai/dzal/YOLOv8/runs/iryqyb0e


Transferred 637/637 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /kaggle/working/IDN-License-Plate-Detection-1/train/labels.cache... 1097 images, 0 backgrounds, 24 corrupt: 100%|██████████| 1097/1097 [00:00<?, ?it/s]


train: WARNING ⚠️ /kaggle/working/IDN-License-Plate-Detection-1/train/images/DataTrain101_jpg.rf.38be56aac2044be7e79f817e36308360.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0026]
train: WARNING ⚠️ /kaggle/working/IDN-License-Plate-Detection-1/train/images/DataTrain108_jpg.rf.b6dee06785157ab22f3edb81349a4a86.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0237      1.0343]
train: WARNING ⚠️ /kaggle/working/IDN-License-Plate-Detection-1/train/images/DataTrain121_jpg.rf.a5ad92f968b691ee4a5eb427707ffd31.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0238      1.0021]
train: WARNING ⚠️ /kaggle/working/IDN-License-Plate-Detection-1/train/images/DataTrain130_jpg.rf.670138cb087b0b0bc2a9fd37def3d86f.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0076]
train: WARNING ⚠️ /kaggle/working/IDN-License-Plate-Detection-1/train/images/DataTra

...

      49/49      7.85G     0.3682     0.2491      1.988          1        640: 100%|██████████| 68/68 [00:45<00:00,  1.49it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:06<00:00,  5.88it/s]


                   all        297        323      0.933      0.944       0.94      0.754

49 epochs completed in 0.752 hours.
Optimizer stripped from runs/obb/train4/weights/last.pt, 139.4MB
Optimizer stripped from runs/obb/train4/weights/best.pt, 139.4MB

Validating runs/obb/train4/weights/best.pt...
Ultralytics YOLOv8.2.46 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
YOLOv8x-obb summary (fused): 287 layers, 69450054 parameters, 0 gradients, 263.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:06<00:00,  5.44it/s]


                   all        297        323      0.941      0.944      0.943      0.757
Speed: 0.1ms preprocess, 14.8ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs/obb/train4


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃▆███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:                  lr/pg1 ▃▆███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:                  lr/pg2 ▃▆███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:        metrics/mAP50(B) ▄▅▄▆▇▃▄▄▄▅▆▄▅▁▄▇▅▄▆▃▄▅▄▆▄▄▄▆▆▅▆▄▅▆▇▇█▇▇█
wandb:     metrics/mAP50-95(B) ▅▅▂▃▄▂▃▁▅▂▆▄▅▄▁▆▄▅▅▄▄▄▄▅▅▄▄▆▅▆▆▅▄▆▇██▇▆█
wandb:    metrics/precision(B) ▅▄▂▄▅▂▅▁▃▆▄▁▂▃▅▇▆█▇▄▆▅▅▃▅▆▄▅▆▆▆▆▇▆▆▆▇▆▅▇
wandb:       metrics/recall(B) ▅▄▄▂▅▁▂▆▇��█▆▃▃▆▄▇▃▂▇▆▆▃█▇▆██▆▆▇▇▅▆▆▆▇▆▇▇
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss ▅▆▇██▇▇▇▇▇▇▆▇▆▆▇▇▆▆▅▅▆▆▅▅▅▅▅▅▅▄▄▂▂▃▂▂▁▁▁
wandb:          train/cls_loss ▅▆▇█████▇▇▇▆▇▆▆▆▆▆▆▅▅▆▆▅▅▅▅▅▅▄▄▄▂▂▃▂▁▁▁▁
wandb:          train/dfl_loss ▅▆▇▇▇█▇▇█▇█▇▆▆▇▆▆▆▆▆▆▅▇▅▆▅▅▅▅▆▅▅▁▂▁▂▂▁▂▁
wandb:            val/box_loss ▃▅█▆▆▅▅▇▅▅▃▃▅▄▅▅▃▃▅▆▃▄▃▂▃▃▂▂▂▂▂▂▃▂

In [122]:
wandb.init(project="YOLOv8-Base-Training", name="YOLOv8 Base Training")
base_model = YOLO("yolov8x.yaml").load("yolov8x.pt")

result = base_model.train(
    data=f"{dataset.location}/data.yaml", 
    epochs=100, 
    imgsz=640, 
    device=[0, 1],
    single_cls=True,
    project="YOLOv8-Base-Training"  # Specify W&B project name
)

Transferred 595/595 items from pretrained weights
Ultralytics YOLOv8.2.46 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.yaml, data=/kaggle/working/IDN-License-Plate-Detection-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=YOLOv8-Base-Training, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_n

wandb: Currently logged in as: dzalhaqi (dzal). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240629_173458-oryluij3
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train2
wandb: ⭐️ View project at https://wandb.ai/dzal/YOLOv8-BASE-TRAINING
wandb: 🚀 View run at https://wandb.ai/dzal/YOLOv8-BASE-TRAINING/runs/oryluij3


Overriding model.yaml nc=80 with nc=1
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /kaggle/working/IDN-License-Plate-Detection-1/train/labels.cache... 1097 images, 0 backgrounds, 24 corrupt: 100%|██████████| 1097/1097 [00:00<?, ?it/s]


train: WARNING ⚠️ /kaggle/working/IDN-License-Plate-Detection-1/train/images/DataTrain101_jpg.rf.38be56aac2044be7e79f817e36308360.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0026]
train: WARNING ⚠️ /kaggle/working/IDN-License-Plate-Detection-1/train/images/DataTrain108_jpg.rf.b6dee06785157ab22f3edb81349a4a86.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0237      1.0343]
train: WARNING ⚠️ /kaggle/working/IDN-License-Plate-Detection-1/train/images/DataTrain121_jpg.rf.a5ad92f968b691ee4a5eb427707ffd31.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0238      1.0021]
train: WARNING ⚠️ /kaggle/working/IDN-License-Plate-Detection-1/train/images/DataTrain130_jpg.rf.670138cb087b0b0bc2a9fd37def3d86f.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0076]
train: WARNING ⚠️ /kaggle/working/IDN-License-Plate-Detection-1/train/images/DataTra

...

    100/100      7.71G     0.4782     0.2603     0.9819          1        640: 100%|██████████| 68/68 [01:32<00:00,  1.35s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:05<00:00,  3.45it/s]


                   all        297        323      0.967       0.99      0.992      0.831

100 epochs completed in 2.802 hours.
Optimizer stripped from YOLOv8-Base-Training/train2/weights/last.pt, 136.7MB
Optimizer stripped from YOLOv8-Base-Training/train2/weights/best.pt, 136.7MB

Validating YOLOv8-Base-Training/train2/weights/best.pt...
Ultralytics YOLOv8.2.46 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
YOLOv8x summary (fused): 268 layers, 68124531 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  2.84it/s]


                   all        297        323      0.967      0.991      0.992      0.831
Speed: 0.1ms preprocess, 14.1ms inference, 0.0ms loss, 3.9ms postprocess per image
Results saved to YOLOv8-Base-Training/train2


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:                  lr/pg1 ▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:                  lr/pg2 ▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:        metrics/mAP50(B) ▁▁▆▇▇███████████████████████████████████
wandb:     metrics/mAP50-95(B) ▁▁▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇█████████████████
wandb:    metrics/precision(B) ▁▂▇▇▇███████████████████████████████████
wandb:       metrics/recall(B) ▃▁▂▅▇▇▇▇▇��██████▇███████████████████████
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▅▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          train/cls_loss █▆▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          train/dfl_loss █▅▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:            val/box_loss █▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁

In [121]:
wandb.finish()